In [1]:
import numpy as np
import ncnn
import joblib

from sklearn.datasets import fetch_openml
from sklearn.metrics.pairwise import cosine_similarity

from skl2onnx import to_onnx
from skl_mlp2ncnn import convert

In [2]:
# load data
X, y = fetch_openml(
    name="mnist_784",
    version=1,
    return_X_y=True,
    as_frame=False,
    parser="pandas",
)

X = X / 255.0

In [4]:
# load model and infer with scikit-learn
autoencoder = joblib.load("mnist_autoencoder.pkl")
X_pred_skl = autoencoder.predict(X[:1])

In [5]:
# convert to onnx
onx = to_onnx(autoencoder, X[:1])

with open("mnist_autoencoder.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [6]:
# convert to ncnn
ncnn_param, ncnn_bin = convert(
    skl_mlp=autoencoder,
    ncnn="mnist_autoencoder",
    verbose=True
)

Input model is MLPRegressor model.
                                      INFO                                      
--------------------------------------------------------------------------------
    Layer 1: Input
        Input shape: (784,)
--------------------------------------------------------------------------------
    Layer 2: InnerProduct
        Output shape: (64,)
--------------------------------------------------------------------------------
    Layer 3: ReLU
--------------------------------------------------------------------------------
    Layer 4: InnerProduct
        Output shape: (784,)
Done! Model converted to mnist_autoencoder.param and mnist_autoencoder.bin.


In [7]:
print(f"param file: {ncnn_param}")
print(f"bin file: {ncnn_bin}")

param file: mnist_autoencoder.param
bin file: mnist_autoencoder.bin


In [8]:
# load ncnn model
with ncnn.Net() as net:
    net.load_param(ncnn_param)
    net.load_model(ncnn_bin)

    with net.create_extractor() as ex:
        ex.input("input_blob", ncnn.Mat(X[0].astype(np.float32)))
        _, out = ex.extract("layer_1_blob")

X_pred_ncnn = np.array(out).reshape((1, -1))

In [9]:
# cosine similarity
cosine_similarity(X_pred_skl, X_pred_ncnn)

array([[0.99999993]])